	# Author: Alexander Staub
	## Last changed: 2025.02.04
	## Purpose: Trialing the Chartmetrci API

In [1]:
#installing packages
import requests
from pprint import pprint
import pandas as pd
import time #for sleep timer

In [2]:
# Define API host and your refresh token
HOST = 'https://api.chartmetric.com'
with open("chartmetric_refresh_token.txt", "r") as f:
    REFRESH_TOKEN = f.read().strip()

In [3]:
# Retrieve an access token using the refresh token
token_response = requests.post(f'{HOST}/api/token', json={'refreshtoken': REFRESH_TOKEN})
if token_response.status_code != 200:
    raise Exception(f"Error: received {token_response.status_code} from /api/token")
access_token = token_response.json()['token']

In [4]:
# Setup headers with the retrieved access token
headers = {'Authorization': f'Bearer {access_token}'}

def get_request(endpoint, params=None):
    response = requests.get(f"{HOST}{endpoint}", headers=headers, params=params)
    if response.status_code != 200:
        raise Exception(f"Error: received {response.status_code} when accessing {endpoint}")
    return response.json()

# Getting information on most popular playlists per platform

The goal is to get the names and Ids of the (100) most popular playlists by platform
Then use the chartmetric ID of the platforms to get the songs on each playlist by dates 
then use the song level information to collect information on artist and performance of the song

In [8]:
#trial without the loop with a single playlist from spotify

#setting the search endpoint
playlist_search_endpoint = '/api/playlist/spotify/lists'

#setting the parameters
params = {'sortColumn': 'followers', 'limit': 10}

#make the request
spotify_top_playlists = get_request(playlist_search_endpoint, params)

#print the results
pprint(spotify_top_playlists)

{'obj': [{'active_ratio': 0.98,
          'activites': None,
          'activities': [{'id': 504878,
                          'isParentTag': False,
                          'name': 'urban',
                          'rank': 1},
                         {'id': 504876,
                          'isParentTag': False,
                          'name': 'partying',
                          'rank': 2},
                         {'id': 504871,
                          'isParentTag': False,
                          'name': 'bonding',
                          'rank': 3},
                         {'id': 504873,
                          'isParentTag': False,
                          'name': 'daydreaming',
                          'rank': 4},
                         {'id': 504872,
                          'isParentTag': False,
                          'name': 'dancy',
                          'rank': 5},
                         {'id': 504886,
                          'isParentTag': Fa

In [ ]:
#defining the search endpoint
playlist_search_endpoint_template = '/api/playlist/{platform}/lists'

#list of platforms to loop over
list_of_platforms = ['spotify', 'deezer', 'amazon', 'youtube', 'itunes']

#list of the sort columns that correspond to each platform
list_of_sort_columns = ['followers', 'followers', 'rank', 'views', 'active_ratio']

#list of whether a playlist should be editorial or not
list_of_editorials = [False, False, False, False, True]

# Create an empty list to store all playlists with platform info
responses_list = []

#UNTESTED

for platform, sort_column, editorial in zip(list_of_platforms, list_of_sort_columns, list_of_editorials):
    # Insert the platform value in the endpoint
    endpoint = playlist_search_endpoint_template.format(platform=platform)
    
    # Set parameters for this request (fetch top 100 playlists)
    params = {'sortColumn': sort_column, 'limit': 100, 'editorial': editorial}
    
    print(f"Requesting top playlists for platform: {platform}")
    response = get_request(endpoint, params)
    
    # Store the raw response along with the platform and request parameters for later parsing
    responses_list.append({
        'platform': platform,
        'sortColumn': sort_column,
        'editorial': editorial,
        'response': response
    })
    
    # Limit request rate to 1 per second
    time.sleep(1)

print("Finished fetching all responses.")
pprint(responses_list)

BELOW HAS NOT BEEN TESTED YET

In [ ]:
# Step 2: Parse the stored JSON responses into a list of playlists with platform info

all_playlist_results = []

for item in responses_list:
    platform = item['platform']
    response = item['response']
    
    # Adjust this parsing based on the actual structure of your JSON response.
    # Here we assume the playlist data is stored under:
    # response['obj']['playlists'][platform]
    playlists = response.get('obj', {}).get('playlists', {}).get(platform, [])
    
    for playlist in playlists:
        # Add the platform info to the playlist record for later filtering.
        playlist['platform'] = platform
        all_playlist_results.append(playlist)

# Optionally, inspect a few results before creating the table.
pprint(all_playlist_results[:2])

# ---------------------------------------------------------------------
# Step 3: Convert the extracted results into a DataFrame and save as CSV

df = pd.DataFrame(all_playlist_results)
df.to_csv('top_playlists_by_platform.csv', index=False)
print("CSV file saved as top_playlists_by_platform.csv")